# Automating the Digitization of Drawn Figures on Maps

### Libraries/Dependencies used

**OpenCV** for color extraction and image detection <br>
**Numpy** for numbers and array manipulation <br>
**glob, ntpath** for directory navigation <br>
**os** for environment variable use <br>
**azureml** for workspace connectivity and dataset creation and access

In [12]:
import numpy as np
import glob
import cv2
import ntpath as nt
import os

### Subscription Information & Datastore Creation

Necessary for workspace-storage connectivity and import of data

Documentation: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data



In [4]:
#Access to subscription information
sub_id = os.getenv("SUBSCRIPTION_ID", default="YOUR_SUBSCRIPTION_ID")
rsc_group = os.getenv("RESOURCE_GROUP", default="YOUR_RESOURCE_GROUP")
ws_name = os.getenv("WORKSPACE_NAME", default="YOUR_CURRENT_WORKSPACE")
ws_region = os.getenv("WORKSPACE_REGION", default="eastus2")

#Access to storage information
azure_storage_account_name = "YOUR_STORAGE_ACCOUNT_NAME"
azure_storage_account_key = "YOUR_STORAGE_ACCOUNT_KEY"

In [5]:
#Connecting to a functional workspace 
from azureml.core import Workspace
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

try:
    ws = Workspace(subscription_id = sub_id, resource_group = rsc_group, workspace_name = ws_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")


# create file dataset from files in datastore
datastore = Datastore.get(ws, 'NAME_OF_THE_DATASTORE')
datastore_path = DataPath(datastore)
file_dataset = Dataset.File.from_files(path=datastore_path)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ANTQZTA3V to authenticate.
Interactive authentication successfully completed.
Workspace configuration succeeded. Skip the workspace creation steps below


In [6]:
#Download the data into the AzureML compute instance. raw_data is an example folder name, you can create/name it whatever you feel like.
file_list = file_dataset.download(target_path="raw_data")

## Feature Extraction

For this project, we will be extracting drawings based on color using **OpenCV**. <br>
OpenCV loads images as *numpy* arrays, consisting of RGB values in a 2D matrix. <br>
Our goal is extracting the pixels that fall within our specified RGB range (BGR as OpenCV formats it this way) and use them to create a mask of the original image.

In [7]:
#Simple directory navigation function, simplifies accessing a file's name from it's directory
def path_leaf(path):
    head, tail = nt.split(path)
    return tail

In [17]:
mask_list = []
def color_masking():
    
    
    #Iterating over the test files, would iterate over repo in the future
    for filename in glob.glob("./raw_data/*.jpg"):

        image = cv2.imread(filename)
        i = 1

        #RGB boundaries for the opencv function of inRange.
        #These values are numpy arrays stored in reverse (BGR)
        #And represent the limits of what we consider a color. In this case, boundaries[0] = red, boundaries[1] = blue and boundaries[2] = green.
        boundaries = [
        ([17, 15, 100], [100, 106, 250]),
        ([86, 31, 4], [220, 88, 50]),
        ([57, 64, 36], [105, 125, 58])
        ]

        for (lower, upper) in boundaries:
            lower = np.array(lower, dtype = "uint8")
            upper = np.array(upper, dtype = "uint8")

            mask = cv2.inRange(image, lower, upper)
            mask_list.append(mask)
            output = cv2.bitwise_and(image, image, mask = mask)

            newfile = path_leaf(filename)
            resultFile = newfile.split(".")
            
            #Depending on the iteration of the loop, the extracted colors vary, and as such the output image must be labelled accordingly.
            #This conditional statement serves that purpose. 
            if i == 1:
                cv2.imwrite("./mask_output/" + resultFile[0] + "_red.jpg", output)
            elif i == 2:
                cv2.imwrite("./mask_output/" + resultFile[0] + "_blue.jpg", output)
            elif i == 3:
                cv2.imwrite("./mask_output/" + resultFile[0] + "_green.jpg", output)
            i += 1



## Creating the Color-Masked images

In [18]:
color_masking()

## Work in Progress:  Coordinate Referencing & Mask Clean Up

The project is nearing completion, missing the coordinate transformation (from pixel coordinates to GPS coordinates) within the actual range of our map's coordinates.
The step after implies the processing of our masks, mainly to eliminate noise (drawings outside the map). <br>

Next Steps:
- Accurately match the masks' features with referenced spatial coordinates (Azure Maps Image Layering)
- Clean up of noise (Marking the valid are and eliminating pixels outside of it)
- Possible: Implementation of Custom Vision model to differentiate between shapes' and their meaning (crosses from polygons)

In [ ]:
# Test of coordinates being available
mask_sample = mask_list[0]

print("Rows: ", len(mask_sample))
print("Columns: ", len(mask_sample[0]))

result = np.where(mask_sample != 0)
listOfCoordinates= list(zip(result[0], result[1]))
print(len(listOfCoordinates))
for cord in listOfCoordinates:
    print(cord)

LinkedIn: https://www.linkedin.com/in/gcordidoa/